# 準備

## artのインストール

In [51]:
!pip3 install adversarial-robustness-toolbox

## ライブラリのインポート

In [52]:
import random
import numpy as np
import matplotlib.pyplot as plt

# TensorFlow with Keras.
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D
from tensorflow.keras.layers import MaxPooling2D, GlobalAveragePooling2D, Dropout
tf.compat.v1.disable_eager_execution()

# ART
import art
from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import KerasClassifier

## Fashion MNISTのロード・前処理

In [76]:
# Fashion MNISTのロード。
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

# Fashion MNISTのラベル。
classes = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
num_classes = len(classes)

In [77]:
# 正規化。
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

# 4次元に変換
X_train = np.expand_dims(X_train, 3)
X_test = np.expand_dims(X_test, 3)

# ラベルをOne-hot-vector化。
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

## 摂動を作るためのClassifierの作成



### モデル定義

In [80]:
# モデルの定義。
def build_model():
  inputs = Input(shape=(28, 28, 1))
  x = Conv2D(64, (3, 3), padding='SAME', activation='relu')(inputs)
  x = Conv2D(64, (3, 3), padding='SAME', activation='relu')(x)
  x = Dropout(0.25)(x)
  x = MaxPooling2D()(x)

  x = Conv2D(128, (3,3), padding='SAME', activation='relu')(x)
  x = Conv2D(128, (3,3), padding='SAME', activation='relu')(x)
  x = Dropout(0.25)(x)
  x = MaxPooling2D()(x)

  x = Conv2D(256, (3,3), padding='SAME', activation='relu')(x)
  x = Conv2D(256, (3,3), padding='SAME', activation='relu')(x)
  x = GlobalAveragePooling2D()(x)

  x = Dense(1024, activation='relu')(x)
  x = Dropout(0.25)(x)
  y = Dense(10, activation='softmax')(x)

  model = Model(inputs, y)

  model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

  return model

model = build_model()

model.summary()

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_54 (Conv2D)          (None, 28, 28, 64)        640       
                                                                 
 conv2d_55 (Conv2D)          (None, 28, 28, 64)        36928     
                                                                 
 dropout_27 (Dropout)        (None, 28, 28, 64)        0         
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 14, 14, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_56 (Conv2D)          (None, 14, 14, 128)       73856     
                                                           

### 学習の実行

In [81]:
# 学習の実行。
model.fit(X_train, y_train,
          batch_size=512,
          epochs=5,
          validation_data=(X_test, y_test),
          shuffle=True)

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - ETA: 0s - loss: 1.0395 - accuracy: 0.5990

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


60000/60000 [==============================] - 35s 589us/sample - loss: 1.0395 - accuracy: 0.5990 - val_loss: 0.6927 - val_accuracy: 0.7598
Epoch 2/5
60000/60000 [==============================] - 30s 500us/sample - loss: 0.5452 - accuracy: 0.7946 - val_loss: 0.5917 - val_accuracy: 0.8160
Epoch 3/5
60000/60000 [==============================] - 30s 506us/sample - loss: 0.4267 - accuracy: 0.8419 - val_loss: 0.4623 - val_accuracy: 0.8491
Epoch 4/5
60000/60000 [==============================] - 30s 498us/sample - loss: 0.3692 - accuracy: 0.8624 - val_loss: 0.4125 - val_accuracy: 0.8648
Epoch 5/5
60000/60000 [==============================] - 30s 507us/sample - loss: 0.3201 - accuracy: 0.8817 - val_loss: 0.3899 - val_accuracy: 0.8765


### モデルの精度評価

In [82]:
model.evaluate(X_test, y_test)

[0.38991572132110597, 0.8765]

## Assistive Signalsの作成

In [83]:
# 入力データの特徴量の最小値・最大値を指定。
# 特徴量は0.0～1.0の範囲に収まるように正規化しているため、最小値は0.0、最大値は1.0とする。
min_pixel_value = 0.0
max_pixel_value = 1.0

# モデルをART Keras Classifierでラップ。
classifier = KerasClassifier(model=model, clip_values=(min_pixel_value, max_pixel_value), use_logits=False)

In [84]:
# FGSMインスタンスの作成。
attack_targeted = FastGradientMethod(estimator=classifier, eps=0.10, targeted=True)
# 敵対的サンプルの生成（ベース画像はテストデータとする）。
X_test_assistive = attack_targeted.generate(x=X_test, y=y_test)

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


## 非標的型攻撃としてのFGSMによる摂動を作成
以下、この摂動を上乗せした画像のことを「敵対的画像」と呼ぶ。

In [85]:
attack_notargeted = FastGradientMethod(estimator=classifier, eps=0.10, targeted=False)
X_test_adv = attack_notargeted.generate(X_test)

## 敵対的画像に対するモデルの精度評価

### cleanのみで学習したモデルとAssistive Signals

In [86]:
model.evaluate(X_test_assistive, y_test)

[0.3115267535209656, 0.9447]

### cleanのみで学習したモデルと敵対的攻撃画像

In [87]:
model.evaluate(X_test_adv, y_test)

[2.4851580669403077, 0.1588]

# Assistive Signals入りの画像のみで学習

In [88]:
X_train_assistive = attack_targeted.generate(x=X_train, y=y_train)

In [89]:
model2 = build_model()

In [90]:
model2.fit(X_train_assistive, y_train,
          batch_size=512,
          epochs=5,
          shuffle=True)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 30s 492us/sample - loss: 0.8529 - accuracy: 0.6758
Epoch 2/5
60000/60000 [==============================] - 30s 497us/sample - loss: 0.1287 - accuracy: 0.9585
Epoch 3/5
60000/60000 [==============================] - 30s 498us/sample - loss: 0.0505 - accuracy: 0.9841
Epoch 4/5
60000/60000 [==============================] - 30s 503us/sample - loss: 0.0339 - accuracy: 0.9893
Epoch 5/5
60000/60000 [==============================] - 30s 492us/sample - loss: 0.0325 - accuracy: 0.9894


In [91]:
# generate assistive X_test on model2
classifier2 = KerasClassifier(model=model2, clip_values=(min_pixel_value, max_pixel_value), use_logits=False)
attack_targeted2 = FastGradientMethod(estimator=classifier2, eps=0.10, targeted=True)
X_test_assistive2 = attack_targeted2.generate(x=X_test, y=y_test)

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


In [96]:
# generate attacked X_test on model2
attack_notargeted2 = FastGradientMethod(estimator=classifier2, eps=0.10, targeted=False)
X_test_adv2 = attack_notargeted2.generate(X_test)

In [92]:
# assistive train data
model2.evaluate(X_train_assistive, y_train)

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


[0.03270250106131037, 0.9935167]

In [93]:
# clean test data
model2.evaluate(X_test, y_test)

[3.3350313636779787, 0.4291]

In [94]:
# assistive test data
model2.evaluate(X_test_assistive2, y_test)

[0.22328898671865463, 0.8995]

In [97]:
# adversarial test data
model2.evaluate(X_test_adv2, y_test)

[3.4411117919921876, 0.303]

# Assistive Signals入り画像とクリーン画像を5:5で学習

In [98]:
X_train_half = list(X_train[:30000])
X_train_half.extend(X_train_assistive[30000:])
X_train_half = np.array(X_train_half)

In [99]:
model3 = build_model()

In [100]:
model3.fit(X_train_half, y_train,
          batch_size=512,
          epochs=5,
          shuffle=True)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 30s 494us/sample - loss: 1.0028 - accuracy: 0.6183
Epoch 2/5
60000/60000 [==============================] - 29s 489us/sample - loss: 0.4023 - accuracy: 0.8583
Epoch 3/5
60000/60000 [==============================] - 30s 496us/sample - loss: 0.2778 - accuracy: 0.9015
Epoch 4/5
60000/60000 [==============================] - 30s 500us/sample - loss: 0.2281 - accuracy: 0.9191
Epoch 5/5
60000/60000 [==============================] - 29s 487us/sample - loss: 0.2005 - accuracy: 0.9284


In [101]:
# generate assistive X_test on model3
classifier3 = KerasClassifier(model=model3, clip_values=(min_pixel_value, max_pixel_value), use_logits=False)
attack_targeted3 = FastGradientMethod(estimator=classifier3, eps=0.10, targeted=True)
X_test_assistive3 = attack_targeted3.generate(x=X_test, y=y_test)

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


In [102]:
# generate attacked X_test on model3
attack_notargeted3 = FastGradientMethod(estimator=classifier3, eps=0.10, targeted=False)
X_test_adv3 = attack_notargeted3.generate(X_test)

In [103]:
# assistive train data
model3.evaluate(X_train_assistive, y_train)

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


[0.060722028885781765, 0.99301666]

In [104]:
# clean test data
model3.evaluate(X_test, y_test)

[0.40575349373817443, 0.8626]

In [105]:
# assistive test data
model3.evaluate(X_test_assistive3, y_test)

[0.04660755504965782, 0.9943]

In [106]:
# adversarial test data
model3.evaluate(X_test_adv3, y_test)

[3.3885780658721925, 0.1412]

# 半分のデータ数で学習

In [117]:
X_train_clean_half = X_train[:30000]
y_train_half = y_train[:30000]

X_train_assistive_half = X_train_assistive[:30000]

In [118]:
model4_clean = build_model()
model4_assistive = build_model()

In [119]:
model4_clean.fit(x=X_train_clean_half, y=y_train_half,
                 epochs=5,
                 batch_size=512,
                 shuffle=True)

Train on 30000 samples
Epoch 1/5
30000/30000 [==============================] - 15s 507us/sample - loss: 1.3822 - accuracy: 0.4630
Epoch 2/5
30000/30000 [==============================] - 15s 494us/sample - loss: 0.7292 - accuracy: 0.7303
Epoch 3/5
30000/30000 [==============================] - 15s 496us/sample - loss: 0.5904 - accuracy: 0.7801
Epoch 4/5
30000/30000 [==============================] - 15s 491us/sample - loss: 0.5439 - accuracy: 0.7997
Epoch 5/5
30000/30000 [==============================] - 15s 495us/sample - loss: 0.4757 - accuracy: 0.8256


In [112]:
model4_assistive.fit(x=X_train_assistive_half, y=y_train_half,
                 epochs=5,
                 batch_size=512,
                 shuffle=True)

Train on 30000 samples
Epoch 1/5
30000/30000 [==============================] - 15s 494us/sample - loss: 1.2871 - accuracy: 0.5055
Epoch 2/5
30000/30000 [==============================] - 15s 488us/sample - loss: 0.3841 - accuracy: 0.8663
Epoch 3/5
30000/30000 [==============================] - 15s 492us/sample - loss: 0.1655 - accuracy: 0.9465
Epoch 4/5
30000/30000 [==============================] - 15s 490us/sample - loss: 0.0884 - accuracy: 0.9703
Epoch 5/5
30000/30000 [==============================] - 15s 488us/sample - loss: 0.0629 - accuracy: 0.9796


In [123]:
model4_clean.evaluate(X_train_clean_half, y_train_half)

[0.48482456364631654, 0.8345]

In [121]:
model4_clean.evaluate(X_test, y_test)

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


[0.5115596541404724, 0.8182]

In [124]:
model4_assistive.evaluate(X_train_assistive_half, y_train_half)

[0.09171677032709122, 0.9762333]

In [125]:
model4_assistive.evaluate(X_test, y_test)

[3.7449867954254152, 0.4441]